# Multi-Modal TSP: Cost Optimization

In [13]:
import os
import requests
import pickle 
import math
import folium
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import googlemaps
import time

# load API keys
load_dotenv()
maps_api_key = os.getenv("MAPS_API_KEY")
amadeus_api_key = os.getenv("AMADEUS_API_KEY")
amadeus_api_secret = os.getenv("AMADEUS_API_SECRET")

gmaps = googlemaps.Client(key=maps_api_key)

def get_amadeus_token():
    response = requests.post(
        'https://test.api.amadeus.com/v1/security/oauth2/token',
        data={
            'grant_type': 'client_credentials',
            'client_id': amadeus_api_key,
            'client_secret': amadeus_api_secret
        }
    )
    return response.json()

In [8]:

locations = ["Vail, CO", "Beaver Creek, CO", "Breckenridge, CO", "Keystone, CO", "A-Basin, CO", "Eldora, CO", "Canyons, UT", "Heavenly, CA", "Northstar, CA", "Kirkwood, CA", "Afton Alps, MN", "Mt. Brighton, MI", "Verbier, Switzerland", "Ski Arlberg, Austria", "The 3 Valleys, France"]
nearest_airport = {
    "Vail, CO": "DEN",
    "Beaver Creek, CO": "DEN",
    "Breckenridge, CO": "DEN",
    "Keystone, CO": "DEN", 
    "A-Basin, CO": "DEN", 
    "Eldora, CO": "DEN", 
    "Canyons, UT": "SLC",
    "Heavenly, CA": "RNO",
    "Northstar, CA": "RNO",
    "Kirkwood, CA": "RNO",
    "Afton Alps, MN": "MSP",
    "Mt. Brighton, MI": "DTW",
    "Verbier, Switzerland": "GVA",
    "Ski Arlberg, Austria": "INNS",
    "The 3 Valleys, France": "GVA"
}


In [9]:
# STEP 1: QUERY APIs

# Query Goolge Maps for specific routes
def get_drive_info(origin, destination):
    try:
        route = gmaps.directions(origin, destination, mode="driving") # Response object from Google API
        print(f"Google Maps call: {origin} → {destination} → {bool(route)}")
        if route:
            leg = route[0]['legs'][0]
            distance_mi = leg['distance']['value'] * 0.000621371 # Convert meters to miles
            duration_hr = leg['duration']['value'] / 3600 # Convert seconds to hours
            return distance_mi, duration_hr
    except Exception as e:
        print(f"Drive error: {origin} → {destination} → {e}")
    return None, None

# Query Amadeus API for flight prices (same as Lauren's code)
def get_flight_price(origin_code, dest_code, departure_date="2025-12-15", adults=1):
    access_token = get_amadeus_token()
    if access_token is None:
        print("Failed to retrieve access token.")
        return None

    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    params = {
        "originLocationCode": origin_code,
        "destinationLocationCode": dest_code,
        "departureDate": departure_date,
        "adults": adults,
        "travelClass": "ECONOMY",
        "currencyCode": "USD",
        "max": 1
    }

    response = requests.get(
        "https://test.api.amadeus.com/v2/shopping/flight-offers",
        headers=headers,
        params=params
    )

    try:
        offers = response.json().get("data", [])
        if offers:
            return float(offers[0]["price"]["total"])
        else:
            print(f"No flight offers found from {origin_code} to {dest_code}.")
    except Exception as e:
        print(f"Error parsing flight price: {e}")
        print("Response JSON:", response.json())
    return None

In [10]:
# STEP 2: BUILD COST MATRIX

def build_cost_matrix(locations, nearest_airport, access_token):
    n = len(locations)
    cost_matrix = np.full((n, n), np.inf)
    mode_matrix = [['' for _ in range(n)] for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            A, B = locations[i], locations[j]

            # Drive
            dist_drive, _ = get_drive_info(A, B)
            cost_drive = dist_drive * 0.5 if dist_drive else np.inf

            if dist_drive is not None and dist_drive < 150:
                cost_matrix[i][j] = cost_drive
                mode_matrix[i][j] = "drive"
                print(f"{A} → {B}: drive = ${cost_drive:.2f}, fly = skipped (short distance)")
                continue

            # Fly
            airport_A, airport_B = nearest_airport[A], nearest_airport[B]
            drive1, _ = get_drive_info(A, airport_A)
            drive2, _ = get_drive_info(airport_B, B)
            flight_price = get_flight_price(airport_A, airport_B, access_token)

            if None not in [drive1, drive2, flight_price]:
                cost_fly = flight_price + 0.5 * (drive1 + drive2)
            else:
                cost_fly = np.inf

            print(f"{A} → {B}: drive = ${cost_drive:.2f}, fly = ${cost_fly:.2f}")

            # Choose cheapest mode
            if cost_drive < cost_fly:
                cost_matrix[i][j] = cost_drive
                mode_matrix[i][j] = "drive"
            else:
                cost_matrix[i][j] = cost_fly
                mode_matrix[i][j] = "fly"

            time.sleep(1)  # API rate limit buffer

    return cost_matrix, mode_matrix

In [17]:
token = get_amadeus_token()
flight_price = get_flight_price("DEN", "GVA", token)
print(f"Flight from DEN to GVA: ${flight_price}")

No flight offers found from DEN to GVA.
Flight from DEN to GVA: $None


In [11]:
# STEP 3: SOLVE TSP (Same as Base TSP)

def solve_tsp(cost_matrix):
    n = len(cost_matrix)
    manager = pywrapcp.RoutingIndexManager(n, 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def cost_callback(from_idx, to_idx):
        from_node = manager.IndexToNode(from_idx)
        to_node = manager.IndexToNode(to_idx)
        return int(cost_matrix[from_node][to_node] * 100)

    transit_callback_idx = routing.RegisterTransitCallback(cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_idx)

    params = pywrapcp.DefaultRoutingSearchParameters()
    params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = routing.SolveWithParameters(params)
    if not solution:
        raise Exception("TSP solution not found")

    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))
    return route

In [14]:
# STEP 4: EVALUATE TRIP

def summarize_trip(route, locations, cost_matrix, mode_matrix):
    total_cost = 0
    for i in range(len(route)-1):
        a, b = route[i], route[i+1]
        origin, dest = locations[a], locations[b]
        cost = cost_matrix[a][b]
        mode = mode_matrix[a][b]
        total_cost += cost
        print(f"{origin} → {dest}: ${cost:.2f} via {mode}")
    print(f"\nTotal trip cost: ${total_cost:.2f}")

# Main Execution
locations = ["Vail, CO", "Beaver Creek, CO", "Breckenridge, CO", "Keystone, CO", "A-Basin, CO", "Eldora, CO", "Canyons, UT", "Heavenly, CA", "Northstar, CA", "Kirkwood, CA", "Afton Alps, MN", "Mt. Brighton, MI", "Verbier, Switzerland", "Ski Arlberg, Austria", "The 3 Valleys, France"]
nearest_airport = {
    "Vail, CO": "DEN",
    "Beaver Creek, CO": "DEN",
    "Breckenridge, CO": "DEN",
    "Keystone, CO": "DEN", 
    "A-Basin, CO": "DEN", 
    "Eldora, CO": "DEN", 
    "Canyons, UT": "SLC",
    "Heavenly, CA": "RNO",
    "Northstar, CA": "RNO",
    "Kirkwood, CA": "RNO",
    "Afton Alps, MN": "MSP",
    "Mt. Brighton, MI": "DTW",
    "Verbier, Switzerland": "GVA",
    "Ski Arlberg, Austria": "INN",
    "The 3 Valleys, France": "GVA"
}

token = get_amadeus_token()
cost_matrix, mode_matrix = build_cost_matrix(locations, nearest_airport, token)
route = solve_tsp(cost_matrix)
summarize_trip(route, locations, cost_matrix, mode_matrix)


Google Maps call: Vail, CO → Beaver Creek, CO → True
Vail, CO → Beaver Creek, CO: drive = $6.21, fly = skipped (short distance)
Google Maps call: Vail, CO → Breckenridge, CO → True
Vail, CO → Breckenridge, CO: drive = $18.54, fly = skipped (short distance)
Google Maps call: Vail, CO → Keystone, CO → True
Vail, CO → Keystone, CO: drive = $19.74, fly = skipped (short distance)
Google Maps call: Vail, CO → A-Basin, CO → True
Vail, CO → A-Basin, CO: drive = $21.14, fly = skipped (short distance)
Google Maps call: Vail, CO → Eldora, CO → True
Vail, CO → Eldora, CO: drive = $48.85, fly = skipped (short distance)
Google Maps call: Vail, CO → Canyons, UT → True
Google Maps call: Vail, CO → DEN → True
Google Maps call: SLC → Canyons, UT → True
No flight offers found from DEN to SLC.
Vail, CO → Canyons, UT: drive = $209.65, fly = $inf
Google Maps call: Vail, CO → Heavenly, CA → True
Google Maps call: Vail, CO → DEN → True
Google Maps call: RNO → Heavenly, CA → True
No flight offers found from DE

KeyboardInterrupt: 

* Google Maps API Docs: https://developers.google.com/maps/documentation/directions/start

* FlightAPI.io Docs: https://www.flightapi.io/docs

* Google OR-Tools for Routing (TSP): https://developers.google.com/optimization/routing

* Haversine Formula (for backup distance calculations): https://en.wikipedia.org/wiki/Haversine_formula

* Python Requests Library: https://docs.python-requests.org/